# 使用自定义数据集训练PromptCLUE模型

In [1]:
# 引入相应的包 Importing libraries
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import numpy as np
import pandas as pd
from rouge import Rouge
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import time, json
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
# from models.modeling_t5 import T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console
print("end2...")

# 计算rouge用
rouge = Rouge()

end2...


In [2]:
# 做一些相关的配置(打印显示；GPU设置)
# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    # console.print(table) # TODO TODO TODO 

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("end...")

end...


# Dataset Class 自定义数据集类

In [3]:
class SmallSampleDataSetClass(Dataset):
    """
    创建一个自定义的数据集，用于训练，必须包括两个字段：输入(如source_text)、输出（如target_text）
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text] if target_text is not None else None
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""
        if self.target_text is not None:
            return len(self.target_text)
        else:
            return len(self.source_text)

    def __getitem__(self, index):
        
        if self.target_text is not None:
            """return the input ids, attention masks and target ids"""

            source_text = str(self.source_text[index])
            target_text = str(self.target_text[index])

            # cleaning data so as to ensure data is in string type
            source_text = " ".join(source_text.split())
            target_text = " ".join(target_text.split())

            source = self.tokenizer.batch_encode_plus(
                [source_text],
                max_length=self.source_len,
                pad_to_max_length=True,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            target = self.tokenizer.batch_encode_plus(
                [target_text],
                max_length=self.summ_len,
                pad_to_max_length=True,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            source_ids = source["input_ids"].squeeze()
            source_mask = source["attention_mask"].squeeze()
            target_ids = target["input_ids"].squeeze()
            target_mask = target["attention_mask"].squeeze()

            return {
                "source_ids": source_ids.to(dtype=torch.long),
                "source_mask": source_mask.to(dtype=torch.long),
                "target_ids": target_ids.to(dtype=torch.long),
                "target_ids_y": target_ids.to(dtype=torch.long),
            }
        else:
            """return the input ids, attention masks and target ids"""

            source_text = str(self.source_text[index])

            # cleaning data so as to ensure data is in string type
            source_text = " ".join(source_text.split())

            source = self.tokenizer.batch_encode_plus(
                [source_text],
                max_length=self.source_len,
                pad_to_max_length=True,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )
            
            source_ids = source["input_ids"].squeeze()
            source_mask = source["attention_mask"].squeeze()

            return {
                "source_ids": source_ids.to(dtype=torch.long),
                "source_mask": source_mask.to(dtype=torch.long)
            }
print("end...")

end...


# 训练方法 Train

In [4]:
class EMA:
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [5]:
def train(epoch, tokenizer, model, device, loader, optimizer, scheduler, ema):

    """
    用于训练的方法
    Function to be called for training with the parameters passed from main function

    """
    n_gpu = torch.cuda.device_count()

    model.train()
    time1 = time.time()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous() # target, from start to end(except end of token, <EOS>). e.g. "你好吗？"
        lm_labels = y[:, 1:].clone().detach() # target, for second to end.e.g."好吗？<EOS>"
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100 # releted to pad_token and loss. for detail, check here: https://github.com/Shivanandroy/T5-Finetuning-PyTorch/issues/3
        ids = data["source_ids"].to(device, dtype=torch.long) # input. e.g. "how are you?"
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        if n_gpu > 1:
            loss = loss.mean()
            
        # 每100步打印日志
        if _ % 100 == 0 and _ != 0 or _ == len(loader) - 1:
            time2 = time.time()
            print("Step:", _,"epoch: " + str(epoch) + "; loss:{:.4f}; each step's time spent:{:.2f}".format(loss.detach().cpu().numpy(), float(time2-time1) / float(_ + 0.0001)))
            # training_logger.add_row(str(epoch), str(_), str(loss))
            # console.print(training_logger)
            # console.log(f"Step: {_}, epoch: {epoch}; loss: {loss.detach().cpu().numpy()}; each step's time spent:{float(time2-time1) / float(_ + 0.0001)}\n")
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema.update()
        scheduler.step()
print("end...")

end...


# 用于验证的方法 Validate

In [6]:
metric_keys = ['main', 'rouge-1', 'rouge-2', 'rouge-l']

def compute_rouge(source, target, unit='word'):
    """计算rouge-1、rouge-2、rouge-l
    """
    # if unit == 'word':
    #     source = jieba.cut(source, HMM=False)
    #     target = jieba.cut(target, HMM=False)
    source, target = ' '.join(source), ' '.join(target)
    try:
        scores = rouge.get_scores(hyps=source, refs=target)
        return {
            'rouge-1': scores[0]['rouge-1']['f'],
            'rouge-2': scores[0]['rouge-2']['f'],
            'rouge-l': scores[0]['rouge-l']['f'],
        }
    except ValueError:
        return {
            'rouge-1': 0.0,
            'rouge-2': 0.0,
            'rouge-l': 0.0,
        }

    
def compute_metrics(source, target, unit='word'):
    """计算所有metrics
    """
    metrics = compute_rouge(source, target, unit)
    metrics['main'] = (
            metrics['rouge-1'] * 0.2 + metrics['rouge-2'] * 0.4 +
            metrics['rouge-l'] * 0.4
    )
    return metrics


def validate(epoch, tokenizer, model, device, loader, max_length, ema):

    """
    用于验证的方法：输入用于验证的数据，返回模型预测的结果和正确的标签
    Function to evaluate model for predictions

    """
    model.eval()
    ema.apply_shadow()
    predictions = []
    actuals = []
    total_metrics = {k: 0.0 for k in metric_keys}
    count = 0
    
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=max_length, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            
            for i in range(len(preds)):
                metrics = compute_metrics(preds[i], target[i])
                for k, v in metrics.items():
                    total_metrics[k] += v
                count += 1
                
            print("preds: %s\ntarget: %s" % (preds[0], target[0]))
                
            if _ % 16 == 0:
                console.print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
            
    avg_metrics = {k: v / count for k, v in total_metrics.items()}
                
    print(avg_metrics)
    console.log(f"{avg_metrics}\n")
    ema.restore()
            
    return predictions, actuals
print("end...")

end...


# 训练类 Trainer

In [7]:
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedShuffleSplit

# 训练类：整合数据集类、训练方法、验证方法，加载数据进行训练并验证训练过程的效果
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/prompt/"
):
    """
    T5 trainer
    """
    n_gpu = torch.cuda.device_count()
    
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using PromptCLUE model and added a Language model layer on top for generation of prediction.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    # model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"], max_seq_len=model_params["MAX_SOURCE_TEXT_LENGTH"])
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)
    
    if n_gpu > 1:
        model = torch.nn.DataParallel(model)
        # model = model.module.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    # display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size So 94% of the data will be used for training and the rest for validation.
    train_size = 0.9998
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)
    
        
    # 打印数据集相关日志：数据量、训练步数
    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"VALID Dataset: {val_dataset.shape}\n")
    total_train_steps = int((train_dataset.shape[0] * model_params["TRAIN_EPOCHS"]) / model_params["TRAIN_BATCH_SIZE"])
    console.print(f"Total Train Steps: {total_train_steps}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = SmallSampleDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = SmallSampleDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    # optimizer = torch.optim.Adam(
    #     params=model.parameters(), lr=model_params["LEARNING_RATE"]
    # )
    optimizer = torch.optim.AdamW(model.parameters(), model_params["LEARNING_RATE"])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * total_train_steps, num_training_steps=total_train_steps)
    
    ema = EMA(model, 0.992)
    ema.register()

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        # 1) train for one epoch
        train(epoch, tokenizer, model, device, training_loader, optimizer, scheduler, ema)
        
        # 2) save model for each epoch
        console.log(f"[Saving Model]...\n")
        path = os.path.join(output_dir, "model_files")
        model.module.save_pretrained(path)
        tokenizer.save_pretrained(path)

        torch.cuda.empty_cache()
        # 3) evaluating test dataset
        console.log(f"[Initiating Validation]...\n")
        with torch.no_grad(): # add 2022.10.4
            #for epoch in range(model_params["VAL_EPOCHS"]):
            predictions, actuals = validate(epoch, tokenizer, model.module, device, val_loader, model_params["MAX_TARGET_TEXT_LENGTH"], ema)
            final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
            final_df.to_csv(os.path.join(output_dir, "predictions.csv"), encoding='utf8', index=None, sep=',')

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")
print("end...")

end...


In [8]:
# 定义模型的参数 let's define model parameters specific to T5
model_params = {
    "MODEL": "outputs/prompt/trained_models/7/",  # model_type pretrained_models/PromptCLUE-base & outputs/prompt/model_files/
    "TRAIN_BATCH_SIZE": 12,  # training batch size, 8
    "VALID_BATCH_SIZE": 16,  # validation batch size, 8 
    "TRAIN_EPOCHS": 2,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 15e-5,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 500,  # max length of source text, 512
    "MAX_TARGET_TEXT_LENGTH": 420,  # max length of target text,64
    "SEED": 2022,  # set seed for reproducibility
}
print("end...")

end...


In [ ]:
# 训练模型
# 使用 pCLUE:1200000+多任务提示学习数据集 的部分数据
# dataframe必须有2列: 
#   - input: 文本输入
#   - target: 目标输出
df = pd.read_csv('data/prompt/train_prompt.tsv', sep='\t', encoding='utf8')  # 数据量：1200k数据。
# df = df.sample(frac=0.01) # TODO  取消本行代码，如果你需要更多数据训练
print("df.head:",df.head(n=5))
print("df.shape:",df.shape)
# 显存占用说明：如果运行现在显存不足，请使用nvidia-smi查看显存；如果显卡多数被占用了，请重启colab程序
T5Trainer(
    dataframe=df,
    source_text="input",
    target_text="target",
    model_params=model_params,
    output_dir="outputs/prompt/"
)

torch.cuda.empty_cache()
print("end..")

df.head:                                                input  \
0  根据标题和关键词生成文章：_标题：零壹智库|150份报告！把脉保险数字化及保险科技创新与发展...   
1  根据标题和关键词生成文章：_标题：明晚直播|保险科技企业服务专题：保险营销服务赛道的机会与挑...   
2  根据标题和关键词生成文章：_标题：专访熊猫保险科技创始人王刚：碎片化场景与精细化营销是保险科...   
3  根据标题和关键词生成文章：_标题：熊猫保险科技出席2021保险科技创新发展论坛_关键词：4月...   
4  根据标题和关键词生成文章：_标题：匠心深耕+科技驱动，众安引领保险科技行业新格局_关键词：新...   

                                              target  
0  科技与金融的融合正在加速。以大数据、人工智能、生物科技、区块链、物联网等为代表的技术不断成熟...  
1  保险科技市场上，企业服务赛道正在成为新的主角：统计过去一年多的保险科技投融资案例，半数以上的...  
2  日前，熊猫保险科技创始人、CEO王刚出席2021保险科技创新发展论坛暨保险科技管理人年会，并...  
3  （4月16日讯）今日，熊猫保险科技受邀参加2021保险科技创新发展论坛暨保险科技管理人年会。...  
4  去年，新冠疫情的袭来，给各行业带来巨大冲击，促使各行业加速洗牌。后疫情时代更是让处于下半场的...  
df.shape: (160331, 2)


[09:29:10] [Model]: Loading outputs/prompt/trained_models/7/...                                     ]8;id=688566;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\118005448.py\118005448.py]8;;\:]8;id=877467;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\118005448.py#19\19]8;;\
                                                                                                                   

[09:29:17] [Data]: Reading data...                                                                  ]8;id=558307;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\118005448.py\118005448.py]8;;\:]8;id=468461;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\118005448.py#35\35]8;;\
                                                                                                                   

FULL Dataset: (160331, 2)

TRAIN Dataset: (160299, 2)

...

preds: 5月20日，助力科技强省建设全省科技创新大会在合肥举行。副省长肖菊华出席并讲话。他强调，要深入贯彻落实全省科技创新大会精神，大力促进科技与金融深度“联姻”，让科技创新“不差钱”科技人才“不愁钱”。肖菊华指出，科技强省建设必须以科技金融为支撑，要以项目签约为新起点，推动形成多元化、多层次、多渠道的科技创新投入体系，努力实现“纸变钱”。肖菊华强调，科技部门要积极协同金融、财政、金融监管部门加快科技成果转化，提档升级，推动政府引导基金市场化、专业化、规范化发展;科技研发金融机构要按照科技强省任务，加大对科技企业需求和市场规律的分析研判，优化金融产品、服务、知识产权质押融资、科技保险等科技金融产品，不断深化“纸变钱”改革，打造更多资本催化剂，更好发挥科技赋能作用。
target: 5月20日，科技金融服务“滴灌行动”助力科技强省建设重大项目签约活动在汉举行。副省长肖菊华出席，强调深入贯彻落实全省科技创新大会精神，大力促进科技和金融深度“联姻”，让科技创新“不差钱”、科技人才“不愁钱”。肖菊华指出，科技强省建设，科技金融不可或缺、大有作为。要以此次项目签约为新起点，推动形成多元化、多层次、多渠道的科技创新投入体系，加快“纸变钱”“钱生钱”。肖菊华强调，科技部门要积极协同金融、财政等部门，推进科技金融服务“滴灌行动”提档升级，推动政府引导基金市场化、专业化、规范化发展，激励和引导更多社会资金投入高新技术行业和科技中小企业，打造全产业链科技金融服务体系。金融监管部门和金融机构要根据科技强省任务、科技企业需求和市场规律，优化金融产品和服务，大力发展知识产权质押融资、科技保险等科技金融产品，为科技成果“纸变钱”注入更多“资本催化剂”。


Completed 0

preds: 近日，黄浦区金光外滩中心的一处融商务、居住、餐饮、娱乐于一体的现代化建筑群正式对外出租。该栋写字楼位于黄浦区金光外滩中心1号楼2层，建筑面积为2001.5平方米，容积率为2002.10，地上27层，裙房、机电、机电设备房、机电、空调等120间，交付标准为A+级;标准交付的住宅有：生物医药、芯片半导体、人工智能科技等。个人介绍：金光外滩中心由金融行业、互联网行业、科技行业、大数据行业、服务型行业等企业创立，选址在金光外滩中心3号楼5层，计划预算约20亿元。
target: 金光外滩中心总投资约四亿美元，是上海外滩城区唯一一处融商务、居住、餐饮和娱乐等多功能的现代化建筑群，总建筑面积约19万平方米。由波特曼建筑设计事务所负责设计。2001.5~2002.10新东京负责酒店(27层)，裙房，地下室的机电深化设计及其它各区的机电设备房(共约8万平方米)的机电深化设计。租赁面积：120平方--2521平方整层租金：6--8.5元/平方/天(不含税)交付标准：精装修及标准交付【适合业态】：总部办公、生物医药研发、芯片半导体研发，新能源检测，人工智能科技等【个人介绍】本人从事地产行业多年，服务过各行各业公司，金融行业，互联网行业，科技行业，大数据行业，服务型行业等等，从企业创立之初为其选址，计划预算，到后期扩大，提供等等一系列服务。
{'main': 0.5587540076503827, 'rouge-1': 0.6828565129558951, 'rouge-2': 0.44857827693980235, 'rouge-l': 0.6068784857082066}


[13:54:17] {'main': 0.5587540076503827, 'rouge-1': 0.6828565129558951, 'rouge-2':                   ]8;id=267822;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\132358616.py\132358616.py]8;;\:]8;id=250491;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_18976\132358616.py#85\85]8;;\
           0.44857827693980235, 'rouge-l': 0.6068784857082066}                                                     
                                                                                                                   

Step: 100 epoch: 1; loss:1.3052; each step's time spent:1.22
Step: 200 epoch: 1; loss:1.6905; each step's time spent:1.20
Step: 300 epoch: 1; loss:2.0328; each step's time spent:1.20
Step: 400 epoch: 1; loss:1.8595; each step's time spent:1.19
Step: 500 epoch: 1; loss:2.0246; each step's time spent:1.19
Step: 600 epoch: 1; loss:1.6146; each step's time spent:1.19
Step: 700 epoch: 1; loss:1.7234; each step's time spent:1.19
Step: 800 epoch: 1; loss:1.8933; each step's time spent:1.19
Step: 900 epoch: 1; loss:1.8595; each step's time spent:1.19
Step: 1000 epoch: 1; loss:2.2911; each step's time spent:1.19
Step: 1100 epoch: 1; loss:1.8001; each step's time spent:1.19
Step: 1200 epoch: 1; loss:1.7406; each step's time spent:1.19
Step: 1300 epoch: 1; loss:1.6552; each step's time spent:1.19
Step: 1400 epoch: 1; loss:1.7111; each step's time spent:1.19
Step: 1500 epoch: 1; loss:1.8312; each step's time spent:1.19
Step: 1600 epoch: 1; loss:1.6639; each step's time spent:1.18
Step: 1700 epoch:

# 预测 Test

In [10]:
def testing(tokenizer, model, device, loader, max_length):

    """
    用于预测的方法：输入用于预测的数据，返回模型预测的结果
    Function for predictions

    """
    
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=max_length, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0,
                early_stopping=True
            )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            
            print(_, preds[0])
            if _ % 100 == 0:
                console.print(f'Completed {_}')

            predictions.extend(preds)
    return predictions
print("end...")

end...


In [11]:
# 训练类：整合数据集类、训练方法、验证方法，加载数据进行训练并验证训练过程的效果
def T5Tester(
    dataframe, source_text, model_params, output_dir="./outputs/prompt/"
):
    """
    T5 tester
    """
    # n_gpu = torch.cuda.device_count()
    n_gpu = 1
    
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True
    
    if n_gpu > 0:
        torch.cuda.manual_seed_all(model_params["SEED"])

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using PromptCLUE model and added a Language model layer on top for generation of prediction.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)
    
    if n_gpu > 1:
        model = torch.nn.DataParallel(model)
        # model = model.module.cuda()

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    # dataframe = dataframe[source_text]
    # display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size So 94% of the data will be used for training and the rest for validation.
    test_dataset = dataframe
    
    # 打印数据集相关日志：数据量、训练步数
    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TEST Dataset: {test_dataset.shape}")

    # Creating the Training and Validation dataset for further creation of Dataloader
    testing_set = SmallSampleDataSetClass(
        test_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text=source_text,
        target_text=None
    )

    # Defining the parameters for creation of dataloaders
    test_params = {
        "batch_size": model_params["TEST_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    testing_loader = DataLoader(testing_set, **test_params)
    
    # 3) evaluating test dataset
    console.log(f"[Initiating Prediction]...\n")
    with torch.no_grad(): # add 2022.10.4
        #for epoch in range(model_params["VAL_EPOCHS"]):
        predictions = testing(tokenizer, model, device, testing_loader, model_params["MAX_TARGET_TEXT_LENGTH"])
        final_df = pd.DataFrame({"Generated Text": predictions})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"), encoding='utf8', index=None)

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Prediction Completed.]\n")
    console.print(
        f"""[Prediction] Generation on Testing data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")
print("end...")

end...


In [12]:
# 定义模型的参数 let's define model parameters specific to T5
model_params = {
    "MODEL": "outputs/prompt/model_files/",  # model_type
    "TEST_BATCH_SIZE": 32,  # training batch size, 8
    "MAX_SOURCE_TEXT_LENGTH": 330,  # max length of source text, 512
    "MAX_TARGET_TEXT_LENGTH": 420,  # max length of target text,64
    "SEED": 2022,  # set seed for reproducibility
}
print("end...")

end...


In [13]:
# 训练模型
# 使用 pCLUE:1200000+多任务提示学习数据集 的部分数据
# dataframe必须有2列: 
#   - input: 文本输入
#   - target: 目标输出
df = pd.read_csv('data/prompt/testB_prompt.tsv', sep='\t', encoding='utf8', header=0, names=["input"])  # 数据量：1675数据。
# df = df.sample(frac=0.01) # TODO  取消本行代码，如果你需要更多数据训练
print("df.head:",df.head(n=5))
print("df.shape:",df.shape)
# 显存占用说明：如果运行现在显存不足，请使用nvidia-smi查看显存；如果显卡多数被占用了，请重启colab程序
T5Tester(
    dataframe=df,
    source_text="input",
    model_params=model_params,
    output_dir="outputs/prompt/prediction/",
)
torch.cuda.empty_cache() 
print("end..")

df.head:                                                input
0  根据标题和关键词生成文章：_标题：统计局：7月份，制造业采购经理指数(PMI)为49.0%_...
1  根据标题和关键词生成文章：_标题：统计局：7月份制造业采购经理指数为49.0%_关键词：统计...
2  根据标题和关键词生成文章：_标题：货币政策这十年：管好总闸门的勇气与探索_关键词：作者;杨志...
3  根据标题和关键词生成文章：_标题：央行：通过结构性货币政策工具等多种政策引导金融机构精准支持...
4  根据标题和关键词生成文章：_标题：7月份中国采购经理指数公布：新动能保持上升势头就业保持相对...
df.shape: (1209, 1)


[23:32:08] [Model]: Loading outputs/prompt/model_files/...                                          ]8;id=649829;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py\877665802.py]8;;\:]8;id=688650;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py#20\20]8;;\
                                                                                                                   

[23:32:15] [Data]: Reading data...                                                                  ]8;id=715946;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py\877665802.py]8;;\:]8;id=17387;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py#35\35]8;;\
                                                                                                                   

FULL Dataset: (1209, 1)

TEST Dataset: (1209, 1)

...

1 习近平在中央统战工作会议上强调促进海内外中华儿女团结奋斗为中华民族伟大复兴汇聚伟力7月29日至30日，中共中央总书记、国家主席、中央军委主席习近平在中央统战工作会议上发表重要讲话。他强调，要坚持以习近平新时代中国特色社会主义思想为指导，坚持中国共产党领导，坚持中国特色社会主义道路和社会主义伟大旗帜，统一思想认识，坚持围绕中心、服务大局，坚定不移推进党和国家事业发展，以优异成绩庆祝建党100周年，推动实现中华民族伟大复兴。习近平指出，党的十八大以来，全国上下始终把人民生命财产安全放在首位，坚持稳中求进工作总基调，全面贯彻新发展理念，不断增强政治定力，努力开创各项工作新局面。
2 据国家统计局网站31日消息，7月份，制造业采购经理指数(PMI)为49.0%，比上月下降1.2个百分点，制造业总体景气水平有所回落。从企业规模看，大型企业PMI为49.5%，比上月下降2.2个百分点，降至临界点以下;中、小型企业PMI分别为47.5%和41.9%，比上月下降0.7个百分点，降至临界点以下。从分类指数看，构成制造业PMI的5个分类指数分别为生产指数、新订单指数、新订单指数和新订单指数。生产指数为49.0%，比上月下降3.0个百分点，降至临界点以下。原材料库存指数为48.4%，比上月下降1.9个百分点，降至临界点以下。新订单指数为48.4%，比上月下降1.9个百分点，降至临界点以下。
3 本文来自：国家统计局7月31日，国家统计局服务业调查中心高级统计师赵庆河解读表示，7月份，制造业采购经理指数(PMI)为49。比上月下降1.2个百分点，位于临界点以下;非制造业商务活动指数为48。综合PMI为48。比上月下降0.9、1.6个百分点，位于临界点以下。总体上看，经济景气水平有所回落，但恢复基础尚需进一步夯实。7月官方制造业PMI为49据统计局数据显示，7月份，制造业采购经理指数为49。比上月下降1.2个百分点，位于临界点以下。从企业规模看，中型企业PMI为49.8%，比上月下降0.4、2.8个百分点，重回临界点以下。生产指数为47.9%，比上月下降0.7个百分点，重回临界点以下。
4 新华社北京6月30日电(王沪宁出席并讲话)新华社电中国国家版本馆开馆暨展览开幕式30日上午在北京举行。中共中央政治局常委、中央书记处书记王沪宁发表讲话，宣布中国国家版本馆正式开馆。王沪宁对党的十八大以

[00:05:28] [Prediction Completed.]                                                                  ]8;id=405267;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py\877665802.py]8;;\:]8;id=950185;file://C:\Users\Administrator\AppData\Local\Temp\ipykernel_12056\877665802.py#79\79]8;;\
                                                                                                                   

[Prediction] Generation on Testing data saved @ outputs/prompt/prediction/predictions.csv

[Logs] Logs saved @ outputs/prompt/prediction/logs.txt

end..


In [ ]:
# 查看训练后显存占用情况。如果显存被占用，可以kill掉相关的进程
!nvidia-smi
# !fuser -v /dev/nvidia*

In [ ]:
# !nvidia-smi -r 
# 使用以下命令清除训练中残存的GPU显存缓存
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache()
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache()  